### Elastic search request api

In [2]:
import json
import logging

import requests


def elasticsearch_curl(uri="http://localhost:9200/", json_body="", verb="get"):
    # pass header option for content type if request has a
    # body to avoid Content-Type error in Elasticsearch v6.0
    headers = {
        "Content-Type": "application/json",
    }
    resp = None
    try:
        # make HTTP verb parameter case-insensitive by converting to lower()
        if verb.lower() == "get":
            resp = requests.get(uri, headers=headers, data=json_body)
        elif verb.lower() == "post":
            resp = requests.post(uri, headers=headers, data=json_body)
        elif verb.lower() == "put":
            resp = requests.put(uri, headers=headers, data=json_body)
        elif verb.lower() == "del":
            resp = requests.delete(uri, headers=headers, data=json_body)
            return None

        # read the text object string
        try:
            resp_text = json.loads(resp.text)
        except:
            resp_text = resp.text

        # catch exceptions and print errors to terminal
    except Exception as error:
        logging.warning("resp:", resp)
        logging.warning("uri:", uri)
        logging.warning("\nelasticsearch_curl() error:", error)
        resp_text = None

    # return the Python dict of the request
    #     print ("resp_text:", resp_text)
    return resp_text


def del_all_scroll():
    response = elasticsearch_curl(
        uri="http://localhost:9200/_search/scroll/_all", verb="del"
    )
    return response


def del_pit(pit):
    json_data = json.dumps({"id": pit})
    response = elasticsearch_curl(
        uri="http://localhost:9200/_pit", json_body=json_data, verb="del"
    )
    return response


response = elasticsearch_curl("http://localhost:9200/wikipedia_sentences/_count")
# print(response)

### Post a sentence to the index

In [3]:
def post_sentence(dict_object):
    # Post one example to news_event_frame index
    if dict_object:
        json_object = json.dumps(dict_object)
    else:
        return None
    response = elasticsearch_curl(
        "http://localhost:9200/wikipedia_sentences/_doc",
        verb="post",
        json_body=json_object,
    )
    return response


def query_sentence(phrase, size=20):
    result = []
    for should_phrase in [
        "is a",
        "is not a",
        "have a",
        "does not have a",
        "is capable of",
        "is not capable of",
    ]:
        dict_object = {
            "query": {
                "bool": {
                    "must": [{"match_phrase": {"doc": phrase}}],
                    "should": [{"match_phrase": {"doc": should_phrase}}],
                },
            },
            "size": size,
        }

        json_object = json.dumps(dict_object)
        response = elasticsearch_curl(
            "http://localhost:9200/wikipedia_sentences/_search",
            verb="post",
            json_body=json_object,
        )
        for item in response["hits"]["hits"]:
            score = item["_score"]
            doc = item["_source"]["doc"]
            result.append((doc, score))
    return result


def query_sentence_match(phrase):
    dict_object = {"query": {"match": {"doc": phrase}}, "size": 1}
    json_object = json.dumps(dict_object)
    response = elasticsearch_curl(
        "http://localhost:9200/wikipedia_sentences/_search",
        verb="post",
        json_body=json_object,
    )
    result = []
    for item in response["hits"]["hits"]:
        score = item["_score"]
        doc = item["_source"]["doc"]
        result.append((doc, score))
    return result


def bulk_post(bulk_dict_data, index_name="wikipedia_sentences"):
    if len(bulk_dict_data) < 1:
        return None
    # Post multiple examples to an index
    # A list of data dict
    meta_json = json.dumps({"index": {"_index": index_name, "_type": "_doc"}})
    data_to_post = (
        "\n".join(meta_json + "\n" + json.dumps(d) for d in bulk_dict_data) + "\n"
    )
    response = elasticsearch_curl(
        f"http://localhost:9200/_bulk", verb="post", json_body=data_to_post,
    )
    return response


# phrase = "entity"
# test_data_dict = [{"query": {"match_phrase": {"doc": phrase}}, "size": 1} for i in range(5)]
# response = post_sentence(test_data_dict)
# # print(response)
# response = bulk_post(test_data_dict)
# print(response)

### Insert sentences into elastic index

In [3]:
data = []
jsonfilename = "../data/LeapOfThought/mask-filling/lot_train.txt"
with open(jsonfilename) as f:
    input_lines = f.readlines()
    for line in input_lines:
        data.append(line)
    print(len(data))
jsonfilename = "../data/LeapOfThought/mask-filling/lot_test.txt"
with open(jsonfilename) as f:
    input_lines = f.readlines()
    for line in input_lines:
        data.append(line)
    print(len(data))
jsonfilename = "../data/LeapOfThought/mask-filling/lot_dev.txt"
with open(jsonfilename) as f:
    input_lines = f.readlines()
    for line in input_lines:
        data.append(line)
    print(len(data))

9793
10158
10510


In [20]:
import random
import string
from random import sample

from tqdm.auto import tqdm

count = lambda l1, l2: sum([1 for x in l1 if x in l2])


def count_punc(s):
    return count(s, set(string.punctuation))


def count_keyword(s, key):
    count = 0
    s = s.replace(".", "").replace("\n", "")
    for w in s.split(" "):
        if w.lower() == key:
            count += 1
    return count


source_obj_set = {}
for item in tqdm(data):
    ## Get source and objective
    sens, obj = item.split("\t")
    obj = obj.strip()
    sen_li = sens.split(". ")
    for sen in sen_li[:-1]:
        sen_ws = sen.split(" ")
        for w in sen_ws:
            if sen.replace(w, "<MASK>") + "." == sen_li[-1]:
                source = w
                source_sent = sen
                break
    ## Get top 10 sentences from wikipedia by source
    #         print(f"source:{source}\n target:{obj}\n source_sent:{source_sent}")
    if source in source_obj_set:
        if obj not in source_obj_set[source]:
            source_obj_set[source].append(obj)
    else:
        source_obj_set[source] = []


source_neural_sent_dic = {}
outfilename = "../data/LeapOfThought/mask-filling/lot_neutral_sent.txt"
with open(outfilename, "w") as f:
    out_sentence = (
        "SOURCE" + "\t" + "NEUTRAL_SENTENCE" + "\t" + "MAX_REPLACE_QUERY_SCORE\n"
    )
    f.write(out_sentence)
    for source, obj_li in tqdm(source_obj_set.items()):
        if len(obj_li) < 1:
            continue
        size = 20
        top_query_sent = query_sentence(
            source, size
        )  # a list of sentences that contain the source word
        top_query_sent = list(set(top_query_sent))
        query_replace_score = []
        ### For each of the 10 sentence, reformulate it by the object word, and then get the one with the lowest similary score in its top-1 result.
        ### This means the word in that sentence is unlikely repalced by the object.
        for sent in tqdm(top_query_sent):
            if len(sent[0].replace("  ", " ").split(" ")) < 4:
                query_replace_score.append(200)
                continue

            if count_punc(sent[0]) > 1:  ## Less than one source word
                query_replace_score.append(100)
                continue

            all_obj_top_scores = []
#             if len(obj_li) > 10:
#                 obj_li = sample(obj_li, 10)
            for obj in obj_li:
                query_replace_sen = sent[0].replace(source, obj)
                top_1_sen, top_1_score = query_sentence_match(query_replace_sen)[0]
                all_obj_top_scores.append(top_1_score)
            query_top_1_score = max(all_obj_top_scores)
            query_replace_score.append(query_top_1_score)
        print(source)
        print(query_replace_score)
        for best_sen, sore in zip(top_query_sent, query_replace_score):
            if sore < 50:
                out_sentence = (
                    source + "\t" + best_sen[0].strip() + "\t" + str(sore) + "\n"
                )
                f.write(out_sentence)
                f.flush()


energy
[100, 100, 82.47045, 100, 100, 109.78167, 80.50147, 100, 52.445667, 53.058426, 100, 100, 100, 48.44327, 39.44076, 37.773136, 67.51372, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 200, 100, 100, 100, 100, 100, 100, 65.54206, 99.83713, 100, 100, 100, 48.115044, 100, 61.4875, 28.394941, 100, 100, 100, 100, 100, 41.178574, 100, 100, 100.97881, 82.80736, 80.153694, 71.74881, 61.822132, 100, 100, 70.74457, 100, 100, 79.43089, 58.32163, 100, 69.48874, 100, 100, 100, 100, 72.78915, 100, 100, 100, 100, 100, 91.025475, 73.572975, 100, 60.294266, 100, 51.28484, 100, 100, 100, 100, 100, 100, 100, 41.062565, 100, 100, 45.81564, 100, 66.716866, 92.07442, 100, 43.132057, 100, 59.403297, 100, 100, 100, 58.893227, 78.97947, 67.33202, 100, 100, 39.05285, 100, 64.78762, 100, 58.00541, 41.278126, 58.589386]


KeyboardInterrupt: 

In [18]:
source

'gentian'

In [19]:
for source, obj_li in tqdm(source_obj_set.items()):
    print(source)

energy
animal
fish
person
element
covering
drug
illumination
appearance
leader
individual
chemical
mammal
man
wood
insect
bark
quantity
food
weather
fabric
nourishment
instrument
platform
vertebrate
traveler
game
sport
football
location
region
trait
material
emotion
building
merchant
vehicle
quality
science
day
delicacy
surface
servant
disorder
barrier
disease
room
attendant
period
activity
road
clothing
decoration
paper
meal
reptile
machine
craft
feeling
ballroom
house
officer
earth
construction
parent
compound
herb
pain
storm
light
church
salamander
commodity
motor
pigeon
education
lamp
mineral
vegetable
cat
joy
art
pup
boat
sound
cloth
bag
dam
book
hair
assistant
garment
formation
gear
precipitation
shape
cutlery
radiation
water
metal
tissue
bread
worker
emperor
job
bottle
corn
coat
biome
bush
medicine
noise
flask
painting
spacecraft
restaurant
table
criminal
official
bone
chicken
shelter
passenger
wave
blubber
solid
tube
server
ground
dish
season
cloud
belief
engine
artist
interval

In [21]:
len(source_obj_set)

326